In [1]:
%run 'common-functions.ipynb'

In [2]:
@dask.delayed
def get_instance_metadata_defaults(client, account_id, account_region):
    result = client.get_instance_metadata_defaults()
    data = {"account_id": [account_id], "region": [account_region], "setting": [result['AccountLevel']]}
    return pd.DataFrame(data)


In [6]:
@dask.delayed
def set_instance_metadata_defaults(client, account_id, account_region):
    result = client.modify_instance_metadata_defaults(
        HttpTokens="required",
        HttpEndpoint="enabled",
        HttpPutResponseHopLimit=2
    )
    return pd.DataFrame(result)

In [3]:
# fetch accounts from metadata once - so dont need to keep taxing ddb in case of issue
custom_filter = (
    Attr("account-type").eq("Connected")
    | Attr("account-type").eq("Foundation")
    | Attr("account-type").eq("Standalone")
    | Attr("account-type").eq("Sandbox")
)
accounts = fetch_accounts_from_metadata(['654654527668'], custom_filter)
print(f"Accounts to process: {len(accounts)}")


Accounts to process: 1318


In [4]:
%%time
# Fetch all cloudwatch log groups across all spokes that dont have a retentionInDays value set
master_df = pd.DataFrame()
dfs = []
for account in accounts:
    account_id = account.get("account")
    account_region = account.get("region")
    if account_region:
        client = assume_return_boto_client("ec2", account_id, account_region)
        dfs.append(get_instance_metadata_defaults(client, account_id, account_region))

graph = dask.delayed()(dfs)
results = graph.compute()
master_df = pd.concat(results)

CPU times: user 53.6 s, sys: 1.77 s, total: 55.3 s
Wall time: 1min 38s


In [5]:
master_df
master_df.to_csv("./h3-imds.csv")